<p>The notebook is based on:  https://www.kaggle.com/jannesklaas/lb-0-6-xgboost-baseline  </p>
<p>Only difference:  xgb_up = XGBClassifier(n_jobs=4,n_estimators=200,max_depth=8,eta=0.1) </p>
<p> His original model: xgb_up = XGBClassifier(n_jobs=4,n_estimators=100) </p>
<p> Please knidly vote  if you like the parameters </p>
    

# XGBoost Baseline

This notebook rephrases the challenge of predicting stock returns as the challenge of predicting whether a stock will go up. The evaluation  asks you to predict a confidence value between -1 and 1. The predicted confidence value gets then multiplied with the actual return. If your confidence is in the wrong direction (ie. you predict positive values while returns are actually negative), you loose on the metric. If your direction is right however, you want your confidence be as large as possible.

Stocks can only go up or down, if the stock is not going up, it must go down (at least a little bit). So if we know our model confidence in the stock going up, then our new confidence is:
$$\hat{y}=up-(1-up)=2*up-1$$

We are left with a "simple" binary classification problem, for which there are a number of good tool, here we use XGBoost, but pick your poison.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import *
import re
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()
(market_train, news_train) = env.get_training_data()

In [ ]:
def data_prep(market_train,news_train):
    market_train.time = market_train.time.dt.date
    news_train.time = news_train.time.dt.hour
    news_train.sourceTimestamp= news_train.sourceTimestamp.dt.hour
    news_train.firstCreated = news_train.firstCreated.dt.date
    news_train['assetCodesLen'] = news_train['assetCodes'].map(lambda x: len(eval(x)))
    news_train['assetCodes'] = news_train['assetCodes'].map(lambda x: list(eval(x))[0])
    news_train['headlineLen'] = news_train['headline'].apply(lambda x: len(x))
    news_train['numberUppercaseLetters'] = news_train['headline'].apply(lambda x: len(re.findall(r'[A-Z]',x)))
    news_train['uppercaseRatio'] = news_train['numberUppercaseLetters'] / news_train['headlineLen']
    kcol = ['firstCreated', 'assetCodes']
    news_train = news_train.groupby(kcol, as_index=False).mean()
    market_train = pd.merge(market_train, news_train, how='left', left_on=['time', 'assetCode'], 
                            right_on=['firstCreated', 'assetCodes'])
    lbl = {k: v for v, k in enumerate(market_train['assetCode'].unique())}
    market_train['assetCodeT'] = market_train['assetCode'].map(lbl)
    
    
    market_train = market_train.dropna(axis=0)
    
    return market_train

In [ ]:
market_train = data_prep(market_train,news_train)

In [ ]:
market_train.head()

In [ ]:
from datetime import datetime, date

In [ ]:
# Remove noisy data from financial crisis of 2008 and before
# Why? Because we have reason to believe this time is significantly different from how it acts now.
market_train = market_train.loc[market_train['time_x']>=date(2009, 1, 1)]

# Now we will feature engineer to improve our score.

In [ ]:
# Open Minus Closing Amounts
market_train['open_minus_close'] = market_train['open'] - market_train['close']
market_train['openprevraw1_minus_closeprevraw1'] = market_train['returnsOpenPrevRaw1'] - market_train['returnsClosePrevRaw1']
market_train['openprevraw10_minus_closeprevraw10'] = market_train['returnsOpenPrevRaw10'] - market_train['returnsClosePrevRaw10']
market_train['openprevmktres1_minus_closeprevmktres1'] = market_train['returnsOpenPrevMktres1'] - market_train['returnsClosePrevMktres1']
market_train['openprevmktres10_minus_closeprevmktres10'] = market_train['returnsOpenPrevMktres10'] - market_train['returnsClosePrevMktres10']

# Differences between opens and closes over lengths of time
market_train['open_change'] = market_train['open'] - market_train['returnsOpenPrevRaw1']
market_train['open_change2'] = market_train['open'] - market_train['returnsOpenPrevRaw10']
market_train['open_change3'] = market_train['open'] - market_train['returnsOpenPrevMktres1']
market_train['open_change4'] = market_train['open'] - market_train['returnsOpenPrevMktres10']
market_train['open_change5'] = market_train['returnsOpenPrevRaw1'] - market_train['returnsOpenPrevRaw10']
market_train['open_change6'] = market_train['returnsOpenPrevRaw1'] - market_train['returnsOpenPrevMktres1']
market_train['open_change7'] = market_train['returnsOpenPrevRaw1'] - market_train['returnsOpenPrevMktres10']
market_train['open_change8'] = market_train['returnsOpenPrevRaw10'] - market_train['returnsOpenPrevMktres1']
market_train['open_change9'] = market_train['returnsOpenPrevRaw10'] - market_train['returnsOpenPrevMktres10']
market_train['open_change10'] = market_train['returnsOpenPrevMktres1'] - market_train['returnsOpenPrevMktres10']

market_train['close_change'] = market_train['close'] - market_train['returnsClosePrevRaw1']
market_train['close_change2'] = market_train['close'] - market_train['returnsClosePrevRaw10']
market_train['close_change3'] = market_train['close'] - market_train['returnsClosePrevMktres1']
market_train['close_change4'] = market_train['close'] - market_train['returnsClosePrevMktres10']
market_train['close_change5'] = market_train['returnsClosePrevRaw1'] - market_train['returnsClosePrevRaw10']
market_train['close_change6'] = market_train['returnsClosePrevRaw1'] - market_train['returnsClosePrevMktres1']
market_train['close_change7'] = market_train['returnsClosePrevRaw1'] - market_train['returnsClosePrevMktres10']
market_train['close_change8'] = market_train['returnsClosePrevRaw10'] - market_train['returnsClosePrevMktres1']
market_train['close_change9'] = market_train['returnsClosePrevRaw10'] - market_train['returnsClosePrevMktres10']
market_train['close_change10'] = market_train['returnsClosePrevMktres1'] - market_train['returnsClosePrevMktres10']

# Verbosity Ratios: Maybe if you have a small word count but large body size, you are using long words -> smarter words -> news impact has more impact?
market_train['sentenceratio'] = market_train['bodySize'] / market_train['sentenceCount']
market_train['wordratio'] = market_train['bodySize'] / market_train['wordCount']
market_train['sentence_to_wordratio'] = market_train['sentenceCount'] / market_train['wordCount']

# Sentiment aggregations
market_train['sentiment_avg'] = np.mean(market_train.head()[['sentimentNegative','sentimentNeutral','sentimentPositive']], axis=1)


In [ ]:
# The target is binary
up = market_train.returnsOpenNextMktres10 >= 0

In [ ]:
fcol = [c for c in market_train if c not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'provider', 
                                             'returnsOpenNextMktres10', 'sourceId', 'subjects', 'time', 'time_x', 'universe','sourceTimestamp']]

In [ ]:
# We still need the returns for model tuning
X = market_train[fcol].values
up = up.values
r = market_train.returnsOpenNextMktres10.values

In [ ]:
# Scaling of X values
# It is good to keep these scaling values for later
mins = np.min(X, axis=0)
maxs = np.max(X, axis=0)
rng = maxs - mins
X = 1 - ((maxs - X) / rng)

In [ ]:
# Sanity check
assert X.shape[0] == up.shape[0] == r.shape[0]

In [ ]:
X_train, X_test, up_train, up_test, r_train, r_test\
= model_selection.train_test_split(X, up, r, test_size=0.25, random_state=99)

In [ ]:
from xgboost import XGBClassifier
import time

In [ ]:
xgb_up = XGBClassifier(n_jobs=4,n_estimators=250,max_depth=8,eta=0.1)

In [ ]:
t = time.time()
print('Fitting Up')
xgb_up.fit(X_train,up_train)
print(f'Done, time = {time.time() - t}')

A side effect of treating this as a binary task is that we can use a simpler metric to judge our models

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(xgb_up.predict(X_test),up_test)

In [ ]:
days = env.get_prediction_days()

In [ ]:
n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days +=1
    print(n_days,end=' ')
    t = time.time()
    market_obs_df = data_prep(market_obs_df, news_obs_df)
    # Open Minus Closing Amounts
    market_obs_df['open_minus_close'] = market_obs_df['open'] - market_obs_df['close']
    market_obs_df['openprevraw1_minus_closeprevraw1'] = market_obs_df['returnsOpenPrevRaw1'] - market_obs_df['returnsClosePrevRaw1']
    market_obs_df['openprevraw10_minus_closeprevraw10'] = market_obs_df['returnsOpenPrevRaw10'] - market_obs_df['returnsClosePrevRaw10']
    market_obs_df['openprevmktres1_minus_closeprevmktres1'] = market_obs_df['returnsOpenPrevMktres1'] - market_obs_df['returnsClosePrevMktres1']
    market_obs_df['openprevmktres10_minus_closeprevmktres10'] = market_obs_df['returnsOpenPrevMktres10'] - market_obs_df['returnsClosePrevMktres10']

    # Differences between opens and closes over lengths of time
    market_obs_df['open_change'] = market_obs_df['open'] - market_obs_df['returnsOpenPrevRaw1']
    market_obs_df['open_change2'] = market_obs_df['open'] - market_obs_df['returnsOpenPrevRaw10']
    market_obs_df['open_change3'] = market_obs_df['open'] - market_obs_df['returnsOpenPrevMktres1']
    market_obs_df['open_change4'] = market_obs_df['open'] - market_obs_df['returnsOpenPrevMktres10']
    market_obs_df['open_change5'] = market_obs_df['returnsOpenPrevRaw1'] - market_obs_df['returnsOpenPrevRaw10']
    market_obs_df['open_change6'] = market_obs_df['returnsOpenPrevRaw1'] - market_obs_df['returnsOpenPrevMktres1']
    market_obs_df['open_change7'] = market_obs_df['returnsOpenPrevRaw1'] - market_obs_df['returnsOpenPrevMktres10']
    market_obs_df['open_change8'] = market_obs_df['returnsOpenPrevRaw10'] - market_obs_df['returnsOpenPrevMktres1']
    market_obs_df['open_change9'] = market_obs_df['returnsOpenPrevRaw10'] - market_obs_df['returnsOpenPrevMktres10']
    market_obs_df['open_change10'] = market_obs_df['returnsOpenPrevMktres1'] - market_obs_df['returnsOpenPrevMktres10']

    market_obs_df['close_change'] = market_obs_df['close'] - market_obs_df['returnsClosePrevRaw1']
    market_obs_df['close_change2'] = market_obs_df['close'] - market_obs_df['returnsClosePrevRaw10']
    market_obs_df['close_change3'] = market_obs_df['close'] - market_obs_df['returnsClosePrevMktres1']
    market_obs_df['close_change4'] = market_obs_df['close'] - market_obs_df['returnsClosePrevMktres10']
    market_obs_df['close_change5'] = market_obs_df['returnsClosePrevRaw1'] - market_obs_df['returnsClosePrevRaw10']
    market_obs_df['close_change6'] = market_obs_df['returnsClosePrevRaw1'] - market_obs_df['returnsClosePrevMktres1']
    market_obs_df['close_change7'] = market_obs_df['returnsClosePrevRaw1'] - market_obs_df['returnsClosePrevMktres10']
    market_obs_df['close_change8'] = market_obs_df['returnsClosePrevRaw10'] - market_obs_df['returnsClosePrevMktres1']
    market_obs_df['close_change9'] = market_obs_df['returnsClosePrevRaw10'] - market_obs_df['returnsClosePrevMktres10']
    market_obs_df['close_change10'] = market_obs_df['returnsClosePrevMktres1'] - market_obs_df['returnsClosePrevMktres10']

    # Verbosity Ratios: Maybe if you have a small word count but large body size, you are using long words -> smarter words -> news impact has more impact?
    market_obs_df['sentenceratio'] = market_obs_df['bodySize'] / market_obs_df['sentenceCount']
    market_obs_df['wordratio'] = market_obs_df['bodySize'] / market_obs_df['wordCount']
    market_obs_df['sentence_to_wordratio'] = market_obs_df['sentenceCount'] / market_obs_df['wordCount']

    # Sentiment aggregations
    market_obs_df['sentiment_avg'] = np.mean(market_obs_df.head()[['sentimentNegative','sentimentNeutral','sentimentPositive']], axis=1)
    market_obs_df = market_obs_df[market_obs_df.assetCode.isin(predictions_template_df.assetCode)]
    X_live = market_obs_df[fcol].values
    X_live = 1 - ((maxs - X_live) / rng)
    prep_time += time.time() - t
    
    t = time.time()
    lp = xgb_up.predict_proba(X_live)
    prediction_time += time.time() -t
    
    t = time.time()
    confidence = 2* lp[:,1] -1
    preds = pd.DataFrame({'assetCode':market_obs_df['assetCode'],'confidence':confidence})
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    env.predict(predictions_template_df)
    packaging_time += time.time() - t

In [ ]:
env.write_submission_file()

In [ ]:
total = prep_time + prediction_time + packaging_time
print(f'Preparing Data: {prep_time:.2f}s')
print(f'Making Predictions: {prediction_time:.2f}s')
print(f'Packing: {packaging_time:.2f}s')
print(f'Total: {total:.2f}s')

For good measure, we can check what XGBoost bases its decisions on

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import plot_importance

In [ ]:
plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
plt.bar(range(len(xgb_up.feature_importances_)), xgb_up.feature_importances_)
plt.xticks(range(len(xgb_up.feature_importances_)), fcol, rotation='vertical');